instalando bibliotecas

In [1]:
pip install findspark

In [2]:
pip install pyspark

importando bibliotecas

In [4]:
import findspark, pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

from pyspark.sql.functions import col, udf

criando instancias

In [5]:
findspark.init()
spark = SparkSession.builder.getOrCreate()

ler e baixar o documento csv

In [16]:
df = spark.read.csv(path="credit-costumers.csv", header=True)
df.show(100)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

modificando a visualização

In [20]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df_csv = spark.read.csv(path="credit-costumers.csv", header=True)
df_csv

CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691,777,11914,1.335,1144,42,1.625,0.061,9.3448e-05,0.99991
818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256,864,7392,1.541,1291,33,3.714,0.105,5.6861e-05,0.99994
713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418,0,3418,2.594,1887,20,2.333,0,2.1081e-05,0.99998
769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313,2517,796,1.405,1171,20,2.333,0.76,0.00013366,0.99987
709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716,0,4716,2.175,816,28,2.5,0,2.1676e-05,0.99998
713061558,Existing Customer,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010,1247,2763,1.376,1088,24,0.846,0.311,5.5077e-05,0.99994
810347208,Existing Customer,51,M,4,Unknown,Married,$120K +,Gold,46,6,1,3,34516,2264,32252,1.975,1330,31,0.722,0.066,0.00012303,0.99988
818906208,Existing Customer,32,M,0,High School,Unknown,$60K - $80K,Silver,27,2,2,2,29081,1396,27685,2.204,1538,36,0.714,0.048,8.5795e-05,0.99991
710930508,Existing Customer,37,M,3,Uneducated,Single,$60K - $80K,Blue,36,5,2,0,22352,2517,19835,3.355,1350,24,1.182,0.113,4.4796e-05,0.99996
719661558,Existing Customer,48,M,2,Graduate,Single,$80K - $120K,Blue,36,6,3,3,11656,1677,9979,1.524,1441,32,0.882,0.144,0.00030251,0.9997


total de meses ordenado pelo maior valor ao menor


In [34]:
df_csv.groupBy("Months_on_book").count().orderBy(col("count").desc()).show()

+--------------+-----+
|Months_on_book|count|
+--------------+-----+
|            36| 2463|
|            37|  358|
|            34|  353|
|            38|  347|
|            39|  341|
|            40|  333|
|            31|  318|
|            35|  317|
|            33|  305|
|            30|  300|
|            41|  297|
|            32|  289|
|            28|  275|
|            43|  273|
|            42|  271|
|            29|  241|
|            44|  230|
|            45|  227|
|            27|  206|
|            46|  197|
+--------------+-----+
only showing top 20 rows



Quantidade de cliente Feminino e Masculino

In [36]:
df_csv.groupBy("Gender").count().orderBy(col("count").desc()).show()

+------+-----+
|Gender|count|
+------+-----+
|     F| 5358|
|     M| 4769|
+------+-----+



Total de clientes com meses inativados


In [37]:
df_csv.groupBy("Months_Inactive_12_mon").count().orderBy(col("count").desc()).show()

+----------------------+-----+
|Months_Inactive_12_mon|count|
+----------------------+-----+
|                     3| 3846|
|                     2| 3282|
|                     1| 2233|
|                     4|  435|
|                     5|  178|
|                     6|  124|
|                     0|   29|
+----------------------+-----+



agrupados por attrition flag

In [40]:
df_csv.groupBy(["Attrition_Flag"]).count()

Attrition_Flag,count
Existing Customer,8500
Attrited Customer,1627


agrupar genero com attrition flag para saber qual genero teve a maior desistencia

In [47]:
df_csv.groupBy("Gender", "Attrition_Flag") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----------------+-----+
|Gender|   Attrition_Flag|count|
+------+-----------------+-----+
|     F|Existing Customer| 4428|
|     M|Existing Customer| 4072|
|     F|Attrited Customer|  930|
|     M|Attrited Customer|  697|
+------+-----------------+-----+

